In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import cv2

# Load the MoveNet Lightning model from TensorFlow Hub
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
movenet = hub.load(model_url)

# Example image path
image_path = 'path_to_your_image.jpg'

# Load and preprocess the image
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_resized = tf.image.resize_with_pad(np.expand_dims(image, axis=0), 192, 192)
input_image = tf.cast(image_resized, dtype=tf.float32)  # Use tf.float32 for input

# Run inference to extract keypoints
outputs = movenet(input_image)
keypoints = outputs['output_0'].numpy()  # Convert output to numpy array
keypoints = np.squeeze(keypoints)

# Scale keypoints to the size of the original image
height, width, _ = image.shape
keypoints[:, 0] *= (width / 192)
keypoints[:, 1] *= (height / 192)

# Draw keypoints on the image
for kp in keypoints:
    x, y, confidence = kp
    if confidence > 0.5:  # Adjust confidence threshold as needed
        cv2.circle(image, (int(x), int(y)), 4, (255, 0, 0), -1)  # Draw a circle at each keypoint location

# Display the image with keypoints
cv2.imshow('Image with Keypoints', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()


2024-07-08 19:35:11.277984: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 19:35:12.292278: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 19:35:13.011876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 19:35:13.721960: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 19:35:13.723985: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 19:35:14.881917: I tensorflow/core/platform/cpu_feature_guard.cc:

TypeError: '_UserObject' object is not callable

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import os
import cv2
import csv
import matplotlib.pyplot as plt

# Load MoveNet Lightning model from TensorFlow Hub
model_url = "https://tfhub.dev/google/movenet/singlepose/lightning/4"
movenet = hub.load(model_url)

print("MoveNet Lightning model loaded successfully.")

MoveNet Lightning model loaded successfully.


In [8]:
def load_and_preprocess_image(image_path):
    # Read the image from file
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Image at path {image_path} could not be read.")
    # Convert image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # Resize the image to maintain aspect ratio and fit within 256x256
    h, w, _ = image.shape
    scale = 256 / max(h, w)
    new_h, new_w = int(h * scale), int(w * scale)
    resized_image = cv2.resize(image, (new_w, new_h))
    # Pad the resized image to be 256x256
    pad_h = (256 - new_h) // 2
    pad_w = (256 - new_w) // 2
    padded_image = cv2.copyMakeBorder(resized_image, pad_h, 256 - new_h - pad_h, pad_w, 256 - new_w - pad_w, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    # Normalize the image to [0,1]
    padded_image = padded_image.astype(np.float32) / 255.0
    return padded_image

In [9]:
# Function to detect keypoints
def detect_keypoints(image):
    input_image = tf.convert_to_tensor(image, dtype=tf.float32)
    input_image = tf.expand_dims(input_image, axis=0)
    input_image = tf.cast(input_image, dtype=tf.int32)  # Convert to int32 as expected by the model
    outputs = movenet.signatures['serving_default'](input_image)
    keypoints = outputs['output_0'].numpy().reshape(-1)
    return keypoints

# Function to save keypoints with class name to CSV
def save_keypoints_with_class(keypoints, class_name, keypoints_path, headers):
    file_exists = os.path.isfile(keypoints_path)
    with open(keypoints_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(headers)
        row = [class_name] + keypoints.tolist()
        writer.writerow(row)

In [10]:
# Function to plot keypoints on image
def plot_keypoints(image, keypoints, threshold=0.5):
    plt.imshow(image)
    h, w, _ = image.shape
    for i in range(0, len(keypoints), 3):
        x, y, conf = keypoints[i:i+3]
        if conf > threshold:  # Only plot keypoints with high confidence
            plt.scatter(x * w, y * h, s=20, c='r', marker='o')  # Adjust coordinates to image scale
    plt.axis('off')
    plt.show()

In [11]:
# Base directory where images are stored
images_base_dir = '../data/images'
# Directory to save keypoints
keypoints_base_dir = '../data/keypoints'
os.makedirs(keypoints_base_dir, exist_ok=True)

# Headers for the CSV file
headers = ["Class",
           "Nose_x", "Nose_y", "Nose_conf",
           "LeftEye_x", "LeftEye_y", "LeftEye_conf",
           "RightEye_x", "RightEye_y", "RightEye_conf",
           "LeftEar_x", "LeftEar_y", "LeftEar_conf",
           "RightEar_x", "RightEar_y", "RightEar_conf",
           "LeftShoulder_x", "LeftShoulder_y", "LeftShoulder_conf",
           "RightShoulder_x", "RightShoulder_y", "RightShoulder_conf",
           "LeftElbow_x", "LeftElbow_y", "LeftElbow_conf",
           "RightElbow_x", "RightElbow_y", "RightElbow_conf",
           "LeftWrist_x", "LeftWrist_y", "LeftWrist_conf",
           "RightWrist_x", "RightWrist_y", "RightWrist_conf",
           "LeftHip_x", "LeftHip_y", "LeftHip_conf",
           "RightHip_x", "RightHip_y", "RightHip_conf",
           "LeftKnee_x", "LeftKnee_y", "LeftKnee_conf",
           "RightKnee_x", "RightKnee_y", "RightKnee_conf",
           "LeftAnkle_x", "LeftAnkle_y", "LeftAnkle_conf",
           "RightAnkle_x", "RightAnkle_y", "RightAnkle_conf"]

# Function to process all images and extract keypoints
def process_images_and_extract_keypoints():
    for class_folder in os.listdir(images_base_dir):
        class_folder_path = os.path.join(images_base_dir, class_folder)
        if os.path.isdir(class_folder_path):
            class_name = class_folder
            image_plotted = False  # Variable to track if an image has been plotted for the current class
            for file in os.listdir(class_folder_path):
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    image_path = os.path.join(class_folder_path, file)
                    keypoints_path = os.path.join(keypoints_base_dir, class_name + '.csv')
                    os.makedirs(os.path.dirname(keypoints_path), exist_ok=True)

                    try:
                        # Load and preprocess image
                        image = load_and_preprocess_image(image_path)
                        # Extract keypoints
                        keypoints = detect_keypoints(image)
                        # Save keypoints with class name to a file
                        save_keypoints_with_class(keypoints, class_name, keypoints_path, headers)
                        print(f"Processed {image_path}, saved keypoints to {keypoints_path}")
                        
                        # Plot keypoints on image for the first image per class
                        if not image_plotted:
                            plot_keypoints(image, keypoints)
                            image_plotted = True  # Set the flag to True after plotting the first image
                            break  # Break out of the loop to ensure only one image is plotted per class
                    except Exception as e:
                        print(f"Error processing {image_path}: {e}")

# Process all images
process_images_and_extract_keypoints()

2024-07-08 18:13:56.453332: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.472597: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.511537: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.532766: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUM

Error processing ../data/images/Upward_Facing_Two-Foot_Staff/Upward_Facing_Two-Foot_Staff_Pose_03.jpeg: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Facing_Two-Foot_Staff/Upward_Facing_Two-Foot_Staff_Pose_39.png: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Facing

2024-07-08 18:13:56.666379: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.695916: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.725560: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.761718: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUM

Error processing ../data/images/Upward_Facing_Two-Foot_Staff/Upward_Facing_Two-Foot_Staff_Pose_104.png: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Facing_Two-Foot_Staff/Upward_Facing_Two-Foot_Staff_Pose_65.png: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Facing

2024-07-08 18:13:56.890849: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.909241: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.961384: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:13:56.993909: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUM

...

2024-07-08 18:14:16.324742: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:14:16.422545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:14:16.458893: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]


Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_101.jpeg: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_66.png: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_59.png: Graph execution error:

Detected at n

2024-07-08 18:14:16.656347: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:14:16.680324: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:14:16.705547: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]]
2024-07-08 18:14:16.794600: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUM

Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_133.jpeg: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_147.jpeg: Graph execution error:

Detected at node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2 defined at (most recent call last):
<stack traces unavailable>
Incompatible shapes: [1,16,16,64] vs. [1,12,12,64]
	 [[{{node center_net_mobile_net_v2fpn_feature_extractor/model_1/tf.__operators__.add/AddV2}}]] [Op:__inference_signature_wrapper_106063]
Error processing ../data/images/Upward_Plank/Upward_Plank_Pose_38.jpeg: Graph execution error:

Detected a

KeyboardInterrupt: 